# IA de Detecção de Tuberculose 
 Detecção de Tuberculose em pacientes através da análise de Raio X

## 1.Importação de Módulos

In [84]:
# Importação de módulos
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras import layers, models
import os
import kagglehub
import shutil
from keras.src.legacy.preprocessing.image import ImageDataGenerator
import PIL

## 2.Base de Dados

In [62]:
# Caminho da pasta do usuário
home_dir = os.path.expanduser("~")

# Caminho onde os datasets do kagglehub são armazenados
dataset_name = "raddar/tuberculosis-chest-xrays-montgomery"
dataset_folder = dataset_name.replace("/", os.sep)  
dataset_path = os.path.join(home_dir, ".cache", "kagglehub", "datasets", dataset_folder, "versions", "1")

# Verifica se o dataset já foi instalado
if not os.path.exists(dataset_path) or not os.listdir(dataset_path):
    print("Baixando o dataset...")
    path = kagglehub.dataset_download("raddar/tuberculosis-chest-xrays-montgomery")
    print("Download concluído! Arquivos em:", path)
else:
    print("Dataset já instalado. Usando arquivos locais em:", dataset_path)

Baixando o dataset...


100%|██████████| 585M/585M [00:35<00:00, 17.1MB/s] 

Extracting files...


Download concluído! Arquivos em: C:\Users\ferna\.cache\kagglehub\datasets\raddar\tuberculosis-chest-xrays-montgomery\versions\1


## 3.Criando IA Convolucional

##### 3.1 Separando casos normais e casos de tuberculose

In [63]:
# Lendo arquivo compilado com os resultados médicos
exams_results = [f for f in os.listdir(dataset_path) if f.endswith('.csv')]
exams_result_path = os.path.join(dataset_path, exams_results[0])

images = os.path.join(dataset_path, 'images', 'images')

In [64]:
# Criando DataFrame
df_exams_results = pd.read_csv(exams_result_path)
df_exams_results

,study_id,age,gender,findings
0,MCUCXR_0001_0.png,27,Female,normal
1,MCUCXR_0002_0.png,40,Female,normal
2,MCUCXR_0003_0.png,21,Female,normal
3,MCUCXR_0004_0.png,11,Female,normal
4,MCUCXR_0005_0.png,33,Male,normal
...,...,...,...,...
133,MCUCXR_0383_1.png,14,Female,calcified granuloma LUL behind 4th rib anterior
134,MCUCXR_0387_1.png,42,Male,inactive scars RUL
135,MCUCXR_0390_1.png,52,Male,Rt apical scarring. LUL scars and some infiltr...
136,MCUCXR_0393_1.png,30,Female,left pleural effusion; best seen on lateral vi...


In [65]:
# Tratando dados
df_exams_results['findings'] = df_exams_results['findings'].str.strip().str.upper()
df_exams_results

,study_id,age,gender,findings
0,MCUCXR_0001_0.png,27,Female,NORMAL
1,MCUCXR_0002_0.png,40,Female,NORMAL
2,MCUCXR_0003_0.png,21,Female,NORMAL
3,MCUCXR_0004_0.png,11,Female,NORMAL
4,MCUCXR_0005_0.png,33,Male,NORMAL
...,...,...,...,...
133,MCUCXR_0383_1.png,14,Female,CALCIFIED GRANULOMA LUL BEHIND 4TH RIB ANTERIOR
134,MCUCXR_0387_1.png,42,Male,INACTIVE SCARS RUL
135,MCUCXR_0390_1.png,52,Male,RT APICAL SCARRING. LUL SCARS AND SOME INFILTR...
136,MCUCXR_0393_1.png,30,Female,LEFT PLEURAL EFFUSION; BEST SEEN ON LATERAL VI...


In [66]:
# Separando imagens normais e patológicas em duas listas
list_normal = []
list_tuberculosis = []

for i, row in df_exams_results.iterrows():
    name_image = row['study_id']
    if row['findings'] == 'NORMAL':
        list_normal.append(name_image)
    else:
        list_tuberculosis.append(name_image)


In [72]:
# Criando novos diretórios para casos normais e casos patológicos
dir_ai_training = 'AI training images'
dir_normal = 'Normal'
dir_tuberculosis = 'Tuberculosis'

path_ai_training = os.path.join(dataset_path, dir_ai_training)
path_normal = os.path.join(dataset_path, dir_ai_training, dir_normal)
path_tuberculosis = os.path.join(dataset_path, dir_ai_training, dir_tuberculosis)

os.makedirs(path_normal, exist_ok=True)
os.makedirs(path_tuberculosis, exist_ok=True)

In [68]:
# Movendo imagens normais e patológicas para seus respectivos diretórios
for name in list_normal:
    origin = os.path.join(images, name)
    destination = os.path.join(path_normal, name)
    if os.path.exists(origin):
        shutil.move(origin, destination)

for name in list_tuberculosis:
    origin = os.path.join(images, name)
    destination = os.path.join(path_tuberculosis, name)
    if os.path.exists(origin):
        shutil.move(origin, destination)

#### 3.2 Pré-Processamento

In [73]:
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = datagen.flow_from_directory(
    path_ai_training,
    target_size=(225,225),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    path_ai_training,
    target_size=(225,225),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

Found 111 images belonging to 2 classes.
Found 27 images belonging to 2 classes.


In [74]:
print(train_generator.class_indices)


{'Normal': 0, 'Tuberculosis': 1}


#### 3.3 Rede Convolucional Neural

In [80]:
model = models.Sequential([
    layers.Conv2D(32, 3, activation='relu', input_shape=(225,225,3)),
    layers.MaxPooling2D(2,2),
    
    layers.Conv2D(64, 3, activation='relu'),
    layers.MaxPooling2D(2,2),
    
    layers.Conv2D(128, 3, activation='relu'),
    layers.MaxPooling2D(2,2),
    
    layers.Flatten(),
    layers.Dropout(0.5),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(1, activation='sigmoid')
])

In [81]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

## 4.Treinando IA

In [85]:
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=10
)

ImportError: Could not import PIL.Image. The use of `load_img` requires PIL.

In [70]:
# Exibindo resultados médicos das imagens fornecidas 
df_exams_results = pd.read_csv(exams_result_path)
df_exams_results

,study_id,age,gender,findings
0,MCUCXR_0001_0.png,27,Female,normal
1,MCUCXR_0002_0.png,40,Female,normal
2,MCUCXR_0003_0.png,21,Female,normal
3,MCUCXR_0004_0.png,11,Female,normal
4,MCUCXR_0005_0.png,33,Male,normal
...,...,...,...,...
133,MCUCXR_0383_1.png,14,Female,calcified granuloma LUL behind 4th rib anterior
134,MCUCXR_0387_1.png,42,Male,inactive scars RUL
135,MCUCXR_0390_1.png,52,Male,Rt apical scarring. LUL scars and some infiltr...
136,MCUCXR_0393_1.png,30,Female,left pleural effusion; best seen on lateral vi...
